In [1]:
#r "nuget:NodaTime,2.4.8"
#r "nuget:Octokit,0.47.0"

using Octokit;
using NodaTime;
using NodaTime.Extensions;
using XPlot.Plotly;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Octokit version 0.47.0

Installed package NodaTime version 2.4.8

In [2]:
var organization = "dfinke";
var repositoryName = "importexcel";
var options = new ApiOptions();
var gitHubClient = new GitHubClient(new ProductHeaderValue("notebook"));


In [3]:
var tokenAuth = new Credentials("8f15d0323fc105b2017671e19714a7b4dcdc648c");
gitHubClient.Credentials = tokenAuth;

In [4]:
var today = SystemClock.Instance.InUtc().GetCurrentDate();
var startOfTheMonth = today.With(DateAdjusters.StartOfMonth);
var startOfPreviousMonth = today.With(DateAdjusters.StartOfMonth) - Period.FromMonths(1);
var startOfTheYear = new LocalDate(today.Year, 1, 1).AtMidnight();

var currentYearIssuesRequest = new RepositoryIssueRequest {
     State = ItemStateFilter.All,
     Since = startOfTheYear.ToDateTimeUnspecified()
};

var pullRequestRequest = new PullRequestRequest {
    State = ItemStateFilter.All
};

In [5]:
#!time
var branches = await gitHubClient.Repository.Branch.GetAll(organization, repositoryName);
var pullRequests = await gitHubClient.Repository.PullRequest.GetAllForRepository(organization, repositoryName, pullRequestRequest);
var forks = await gitHubClient.Repository.Forks.GetAll(organization, repositoryName);
var currentYearIssues = await gitHubClient.Issue.GetAllForRepository(organization, repositoryName, currentYearIssuesRequest);

Wall time: 21376.7409ms

In [6]:

var pullRequestCreatedThisMonth = pullRequests.Where(pr => pr.CreatedAt > startOfTheMonth.ToDateTimeUnspecified());
var pullRequestClosedThisMonth =pullRequests.Where(pr => (pr.MergedAt != null && pr.MergedAt > startOfTheMonth.ToDateTimeUnspecified()));
var contributorsCount = pullRequestClosedThisMonth.GroupBy(pr => pr.User.Login);

var pullRequestLifespan = pullRequests.GroupBy(pr =>
            {
                var lifeSpan = (pr.ClosedAt ?? today.ToDateTimeUnspecified()) - pr.CreatedAt;
                return Math.Max(0, Math.Ceiling(lifeSpan.TotalDays));
            })
            .Where(g => g.Key > 0)
            .OrderBy(g => g.Key)
            .ToDictionary(g => g.Key, g => g.Count());

In [7]:
var forkCreatedThisMonth = forks.Where(fork => fork.CreatedAt >= startOfTheMonth.ToDateTimeUnspecified());
var forkCreatedPreviousMonth = forks.Where(fork => (fork.CreatedAt >= startOfPreviousMonth.ToDateTimeUnspecified()) && (fork.CreatedAt < startOfTheMonth.ToDateTimeUnspecified()));
var forkCreatedByMonth = forks.GroupBy(fork => new DateTime(fork.CreatedAt.Year, fork.CreatedAt.Month, 1));
var forkUpdateByMonth = forks.GroupBy(f => new DateTime(f.UpdatedAt.Year, f.UpdatedAt.Month,  1) ).Select(g => new {Date = g.Key, Count = g.Count()}).OrderBy(g => g.Date).ToArray();
var total = 0;
var forkCountByMonth = forkCreatedByMonth.OrderBy(g => g.Key).Select(g => new {Date = g.Key, Count = total += g.Count()}).ToArray();


In [8]:
bool IsBug(Issue issue){
    return issue.Labels.FirstOrDefault(l => l.Name == "bug")!= null;
}

bool TargetsArea(Issue issue){
    return issue.Labels.FirstOrDefault(l => l.Name.StartsWith("Area-"))!= null;
}

string GetArea(Issue issue){
    return issue.Labels.FirstOrDefault(l => l.Name.StartsWith("Area-"))?.Name;
}

var openIssues = currentYearIssues.Where(IsBug).Where(issue => issue.State == "open");
var closedIssues = currentYearIssues.Where(IsBug).Where(issue => issue.State == "closed");
var oldestIssues = openIssues.OrderBy(issue => today.ToDateTimeUnspecified() - issue.CreatedAt).Take(20);
var createdCurrentMonth = currentYearIssues.Where(IsBug).Where(issue => issue.CreatedAt >= startOfTheMonth.ToDateTimeUnspecified());
var createdPreviousMonth = currentYearIssues.Where(IsBug).Where(issue => (issue.CreatedAt >= startOfPreviousMonth.ToDateTimeUnspecified()) && (issue.CreatedAt < startOfTheMonth.ToDateTimeUnspecified()));
var openFromPreviousMonth = openIssues.Where(issue => (issue.CreatedAt > startOfPreviousMonth.ToDateTimeUnspecified()) && (issue.CreatedAt < startOfTheMonth.ToDateTimeUnspecified()));
var createdByMonth = currentYearIssues.Where(IsBug).GroupBy(issue => new DateTime(issue.CreatedAt.Year, issue.CreatedAt.Month, 1)).OrderBy(g=>g.Key).ToDictionary(g => g.Key, g => g.Count());
var closedByMonth = closedIssues.GroupBy(issue => new DateTime((int) issue.ClosedAt?.Year, (int) issue.ClosedAt?.Month, 1)).OrderBy(g=>g.Key).ToDictionary(g => g.Key, g => g.Count());
var openIssueAge = openIssues.GroupBy(issue => new DateTime(issue.CreatedAt.Year, issue.CreatedAt.Month, issue.CreatedAt.Day)).ToDictionary(g => g.Key, g => g.Max(issue =>Math.Max(0, Math.Ceiling( (today.ToDateTimeUnspecified() - issue.CreatedAt).TotalDays))));
var openByMonth = new Dictionary<DateTime, int>();
var minDate = createdByMonth.Min(g => g.Key);
var maxCreatedAtDate = createdByMonth.Max(g => g.Key);
var maxClosedAtDate = closedByMonth.Max(g => g.Key);
var maxDate = maxCreatedAtDate > maxClosedAtDate ?maxCreatedAtDate : maxClosedAtDate;
var cursor = minDate;
var runningTotal = 0;
var issuesCreatedThisMonthByArea = currentYearIssues.Where(issue => issue.CreatedAt >= startOfTheMonth.ToDateTimeUnspecified()).Where(issue => IsBug(issue) && TargetsArea(issue)).GroupBy(issue => GetArea(issue)).ToDictionary(g => g.Key, g => g.Count());
var openIssueByArea = currentYearIssues.Where(issue => issue.State == "open").Where(issue => IsBug(issue) && TargetsArea(issue)).GroupBy(issue => GetArea(issue)).ToDictionary(g => g.Key, g => g.Count());

while (cursor <= maxDate )
{
    createdByMonth.TryGetValue(cursor, out var openCount);
    closedByMonth.TryGetValue(cursor, out var closedCount);
    runningTotal += (openCount - closedCount);
    openByMonth[cursor] = runningTotal;
    cursor = cursor.AddMonths(1);
}

var issueLifespan = currentYearIssues.Where(IsBug).GroupBy(issue =>
            {
                var lifeSpan = (issue.ClosedAt ?? today.ToDateTimeUnspecified()) - issue.CreatedAt;
                return Math.Max(0, Math.Round(Math.Ceiling(lifeSpan.TotalDays),0));
            })
            .Where(g => g.Key > 0)
            .OrderBy(g => g.Key)
            .ToDictionary(g => g.Key, g => g.Count());

display(new { 
    less_then_one_sprint = issueLifespan.Where(i=> i.Key < 21).Select(i => i.Value).Sum(),
    less_then_two_sprint = issueLifespan.Where(i=> i.Key >= 21 && i.Key < 42).Select(i => i.Value).Sum(),
    more_then_two_sprint = issueLifespan.Where(i=> i.Key >= 42).Select(i => i.Value).Sum()   
    });

less_then_one_sprint,less_then_two_sprint,more_then_two_sprint
1,0,4


In [9]:
var prColors = pullRequestLifespan.OrderBy(pr => pr.Key).Select(pr => pr.Key < 7 ? "green" : pr.Key < 21 ? "yellow" : "red");
var prLifespanOneWeekSeries = new Graph.Bar
{
    name = "One week",
    y = pullRequestLifespan.Where(issue => issue.Key < 7).OrderBy(pr => pr.Key).Select(pr => pr.Value),
    x = pullRequestLifespan.Where(issue => issue.Key < 7).OrderBy(pr => pr.Key).Select(pr => pr.Key) ,
    marker = new Graph.Marker{
        color = "green"
    }  
};

var prLifespanOneSprintSeries = new Graph.Bar
{
    name = "One Sprint",
    y = pullRequestLifespan.Where(issue => issue.Key >= 7 && issue.Key < 21).OrderBy(pr => pr.Key).Select(pr => pr.Value),
    x = pullRequestLifespan.Where(issue => issue.Key >= 7 && issue.Key < 21).OrderBy(pr => pr.Key).Select(pr => pr.Key) ,
    marker = new Graph.Marker{
        color = "yellow"
    }  
};

var prLifespanMoreThanASprintSeries = new Graph.Bar
{
    name = "More than a Sprint",
    y = pullRequestLifespan.Where(issue => issue.Key >= 21).OrderBy(pr => pr.Key).Select(pr => pr.Value),
    x = pullRequestLifespan.Where(issue => issue.Key >= 21).OrderBy(pr => pr.Key).Select(pr => pr.Key) ,
    marker = new Graph.Marker{
        color = "red"
    }  
};

var prLifespanChart = Chart.Plot(new[] {prLifespanOneWeekSeries, prLifespanOneSprintSeries, prLifespanMoreThanASprintSeries});
prLifespanChart.WithLayout(new Layout.Layout
            {
                title = "Pull Request by life span",
                xaxis = new Graph.Xaxis {
                    title = "Number of days a PR stays open",
                    showgrid = false,
                    zeroline = false
                },
                yaxis = new Graph.Yaxis {
                    title = "Number of PR",
                    showgrid = true,
                    zeroline = false
                }
            });
display(prLifespanChart);

In [10]:
var forkCreationSeries = new Graph.Scattergl
{
    name = "created by month",
    y = forkCreatedByMonth.Select(g => g.Count() ).ToArray(),
    x = forkCreatedByMonth.Select(g => g.Key ).ToArray()
};

var forkTotalSeries = new Graph.Scattergl
{
    name = "running total",
    y = forkCountByMonth.Select(g => g.Count ).ToArray(),
    x = forkCountByMonth.Select(g => g.Date ).ToArray()
};

var forkUpdateSeries = new Graph.Scattergl
{
    name = "last update by month",
    y = forkUpdateByMonth.Select(g => g.Count ).ToArray(),
    x = forkUpdateByMonth.Select(g => g.Date ).ToArray()
};



var chart = Chart.Plot(new[] {forkCreationSeries,forkTotalSeries,forkUpdateSeries});
chart.WithTitle("Fork activity");
display(chart)